do some basic pre-processing. For example, the passenger names will be tokenized, and ticket names will be splitted in parts.

train a Gradient Boosted Trees (GBT) with default parameters

train a GBT with improved default parameters

tune the parameters of a GBTs

train and ensemble many GBTs

In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.12.0


In [3]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import ydf 

print(f"Found YDF {ydf.__version__}")

Found YDF 0.11.0


Loading trainig set:

In [16]:
train_df = pd.read_csv("/Users/gaby/Desktop/Repos/titanic/train.csv")
test_df = pd.read_csv("/Users/gaby/Desktop/Repos/titanic/test.csv")

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

In [13]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Preparing data_set:

We will apply the following transformations on the dataset.

Tokenize the names. (For example, "Braund, Mr. Owen Harris" will become ["Braund", "Mr.", "Owen", "Harris"].)
Extract any prefix in the ticket. (For example ticket "STON/O2. 3101282" will become "STON/O2." and 3101282.)

In [ ]:
def preprocess(df):
#let's do a copy of the original dataset    
    df = df.copy()
#Now, let's define functions: 1) to normalize the name of the passenger
#                             2) to get the ticket's number
#                             3) to get the item (if it has a letter o code before the number)
    def normalize_name(x):
        """
        Concatenate any number of strings.
        The string whose method is called is inserted in between each given string. 
        The result is returned as a new string.
        """
        return "".join([v.strip(",()[].\"'" )for v in x.split(" ")])
    
    def ticket_number(x):
        """
        It recieves an string
        returns an string 
        """
        return x.split(" ")[-1]
    
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return 'NONE'
        return "_".join(items[0:-1])
# then, some columns must be added: 1) 'Name'
#                                 2) 'Ticket_number' 
#                                 3) 'Ticket_item'  
    df['Name'] = df["Name"].apply(normalize_name)
    df['Ticket_number'] = df['Ticket'].apply(ticket_number)
    df['Ticket_item'] = df['Ticket'].apply(ticket_item)

    return df

In [ ]:
#tokenize the names

def preprocess(df):
    df = df.copy()

    def normalize_name(x):
        return "".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
    
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
        
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)

    return df


preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(test_df)


preprocessed_train_df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,BraundMrOwenHarris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,CumingsMrsJohnBradleyFlorenceBriggsThayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,HeikkinenMissLaina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,FutrelleMrsJacquesHeathLilyMayPeel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,AllenMrWilliamHenry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE
5,6,0,3,MoranMrJames,male,NaN,0,0,330877,8.4583,NaN,Q,330877,NONE
6,7,0,1,McCarthyMrTimothyJ,male,54.0,0,0,17463,51.8625,E46,S,17463,NONE
7,8,0,3,PalssonMasterGostaLeonard,male,2.0,3,1,349909,21.0750,NaN,S,349909,NONE
8,9,1,3,JohnsonMrsOscarWElisabethVilhelminaBerg,female,27.0,0,2,347742,11.1333,NaN,S,347742,NONE
9,10,1,2,NasserMrsNicholasAdeleAchem,female,14.0,1,0,237736,30.0708,NaN,C,237736,NONE


Let's keep the list of the input features of the model. Notably, we don't want to train our model on the "PassengerId" and "Ticket" features.

In [6]:
inputs_features = list(preprocessed_train_df.columns)

inputs_features.remove("PassengerId")
inputs_features.remove("Ticket")

#Also  we take out the "y" value, known as "Survived", it's not a feature, it is "label"

inputs_features.remove("Survived")

print(f"Input Features: {inputs_features}")
print(len(inputs_features))


Input Features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']
11


Convert Pandas dataset to TensorFlow Dataset

In [7]:
def tokenize_names(features, labels = None):
    """
    Divite the names into tokens. TF-DF can consume text tokens natively.
    """
    features["Name"] = tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label = "Survived").map(tokenize_names)

serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)


Train model with default parameters!!

First, we are training a GradientBoostedTreesModel model with the default parameters.

In [8]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose = 0, #very few logs
    features = [tfdf.keras.FeatureUsage(name = n) for n in inputs_features], 
    exclude_non_specified_features = True,   #only use features in "features"
    random_seed= 1234,  
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")



[WARNING 23-08-16 17:18:31.5313 CEST gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-16 17:18:31.5337 CEST gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-16 17:18:31.5337 CEST gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-08-16 17:18:31.714093: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
[INFO 23-08-16 17:18:35.0068 CEST kernel.cc:1243] Loading model from path /var/folders/yh/nttnvvwn565_gnd98_z427v80000gp/T/tmpnd_kciyk/model/ with prefix a7b337c148f54a39
[INFO 23-08-16 17:18:35.0134 CEST abstract_model.cc:1312] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-16 17:18:35.0135 CEST kernel.cc:1075] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Accuracy: 0.8152173757553101 Loss:0.8874466419219971


Train model with improved default parameters

-->  use some specific parameters when creating the GBT model

In [11]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose= 0,
    features = [tfdf.keras.FeatureUsage(name=n) for n in inputs_features],
    exclude_non_specified_features= True,

    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples = 1,
    categorical_algorithm = "RANDOM",
    #MÁX DEPHT = 4

    shrinkage= 0.05,
     #num_candidate_attributes_ratio=0.2,

    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,

)

model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss: {self_evaluation.loss}")


[WARNING 23-08-16 17:19:24.4793 CEST gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-16 17:19:24.4794 CEST gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-16 17:19:24.4794 CEST gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-16 17:19:25.0140 CEST kernel.cc:1243] Loading model from path /var/folders/yh/nttnvvwn565_gnd98_z427v80000gp/T/tmpcbdy6dw7/model/ with prefix 74ab76e7c2db49b0
[INFO 23-08-16 17:19:25.0212 CEST decision_forest.cc:660] Model loaded with 47 root(s), 2497 node(s), and 9 input feature(s).
[INFO 23-08-16 17:19:25.0213 CEST abstract_model.cc:1312] Engine "GradientBoostedTreesGeneric" built
[INFO 23-08-16 17:19:25.0213 CEST kernel.cc:1075] Use fast generic engine


Accuracy: 0.77173912525177 Loss: 1.0416638851165771


In [12]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.949495 ################
    2.           "Age"  0.382930 ####
    3.          "Fare"  0.268009 #
    4.   "Ticket_item"  0.181725 
    5. "Ticket_number"  0.179416 
    6.      "Embarked"  0.176179 
    7.        "Pclass"  0.175175 
    8.         "Parch"  0.174435 
    9.         "SibSp"  0.171619 

Variable Importance: NUM_AS_ROOT:
    1. "Sex" 46.000000 ################
    2. "Age"  1.000000 

Variable Importance: NUM_NODES:
    1.   

MAKE PREDICTIONS


In [20]:
def prediction_to_kaggle_format(model, threshold = 0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path = "/Users/gaby/Desktop/Repos/titanic/gender_submission.csv"
    kaggle_predictions.to_csv(path, index = False)
    print(f"Submission exported to: {path}")

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)

!head /Users/gaby/Desktop/Repos/titanic/gender_submission.csv



Submission exported to: /Users/gaby/Desktop/Repos/titanic/gender_submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1


>Training a model with hyperparameter tunning

Hyper-parameter tuning is enabled by specifying the tuner constructor argument of the model. The tuner object contains all the configuration of the tuner (search space, optimizer, trial and objective).

In [ ]:
tuner = tfdf.tuner.RandomSearch(num_trials= 1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", [LOCAL])
local_search_space.choice("max_depth", [3, 4, 5, 6, 7, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge = True)

global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

#tuner.choice("use_hessian_gain", [True, False])

tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge = True)
oblique_space.choice("sparse_oblique_normalization", 
                     ["NONE", "STANDARD_DESVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])


